In [18]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import time
from nltk.corpus import wordnet
import re
import enchant
import random
import numpy as np
from langdetect import detect

# Read data

In [3]:
filename = '/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/data/testval_data.csv'
reviews = pd.read_csv(filename)

In [4]:
yelp = reviews.copy()
yelp.shape

(1016664, 8)

# 1 加上长度变量

In [5]:
yelp.loc[:, 'text length'] = yelp['text'].apply(len)

# 2 改缩写和换行符

In [6]:
shortcut_dir = {'n\'t': ' not', '\'s': ' is', '\'ve': ' have', '\'d': ' would', '\'m': ' am',
                'Ive': 'I have', 'cannot': 'can not', '\\n': ' '}

In [7]:
t1 = time.time()
def shortcut_remove(review):
    review = review.lower()
    for i in shortcut_dir:
        review = re.sub(i, shortcut_dir[i], review)
    return review
yelp2 = yelp.copy()
yelp2['text'] = yelp2['text'].apply(shortcut_remove)
time.time() - t1

16.521751165390015

# 3 Delete unEnglish

In [8]:
d = enchant.Dict("en_US")

In [27]:
def unEnglish_remove(review):
    review = re.sub('[^a-zA-Z]',' ', review)
    words = pd.Series(review.split())
    index = words.apply(d.check)
    if(len(index)==0):
        return True
    else:
        return(sum(index)/len(index)<0.8)
t1 = time.time()
index = yelp2['text'].apply(unEnglish_remove)
time.time() - t1

KeyboardInterrupt: 

In [28]:
yelp3 = yelp2.copy()

# 4 Add not

In [29]:
inverter = ['not', 'never', 'lack', 'no']
def add_not(sentence):
    words = sentence.split().copy()
    not_index = 0
    for index, word in enumerate(words):
        if(not_index != 0):
            words[index] = 'not' + words[index]
        if(re.match('.*[,.?!\'\"]$', word)):
            not_index = 0
        if(word in inverter):
            not_index = 1
    #         print(index)
    sentence = " ".join(words)
    return(sentence)
t1 = time.time()
yelp4 = yelp3.copy()
yelp4['text'] = yelp3['text'].apply(add_not)
time.time() - t1

157.39355278015137

# Write file

In [30]:
filename_li = "../../data/test.csv"
yelp4.to_csv(filename_li, index = False)